# Rice Data and Donuts Time Series Workshop Interactive Exercises: Python
####  Corrin Fosmire (Rice University)
#### August 1st, 2019

## Introduction

This notebook will run through a full forecasting modeling session, using real data from the Dow Jones Industrial Average. Let's get started!

## Exploration of Data

Let's begin by taking a look at our data and exploring some relationships.

In [ ]:
import pandas as pd
from plotnine import *
from sklearn.linear_model import LinearRegression
import numpy as np
import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace import sarimax
from fbprophet import Prophet

djia = pd.read_csv("djiafixed.csv")
djia["Date"] = pd.to_datetime(djia["Date"])
djiaclean = djia.fillna(method='ffill')
djia = djia.fillna(method='ffill')

In [ ]:
djiaclean.head()

In [ ]:
(ggplot(djiaclean) +
  geom_line(mapping=aes('Date', 'Close')))

In [ ]:
daygroup = djiaclean.groupby('month').Close.agg('mean').reset_index()
(ggplot(daygroup) +
  geom_point(mapping=aes('month', 'Close')) +
  geom_line(mapping=aes('month', 'Close')) +
  labs(title="Average Dow Jones Close by Month between 2013 and 2018 ", y="DJIA Close", x="Month"))

In [ ]:
daygroup = djiaclean.groupby('yday').Close.agg('mean').reset_index()
(ggplot(daygroup) +
  geom_line(mapping=aes('yday', 'Close')) +
  labs(title="Average Dow Jones Close by Day of Year between 2013 and 2018 ", y="DJIA Close", x="Day of Year"))

In [ ]:
daygroup = djiaclean.groupby('mday').Close.agg('mean').reset_index()
(ggplot(daygroup) +
  geom_line(mapping=aes('mday', 'Close')) +
  labs(title="Average Dow Jones Close by Day of Month between 2013 and 2018 ", y="DJIA Close", x="Day of Month"))

In [ ]:
daygroup = djiaclean.groupby('wday').Close.agg('mean').reset_index()
(ggplot(daygroup) +
  geom_line(mapping=aes('wday', 'Close')) +
  labs(title="Average Dow Jones Close by Day of Week between 2013 and 2018 ", y="DJIA Close", x="Day of Week"))

## Checking out the trend

We first fit an simple line of best fit model, just to give us an idea how the mean is changing over time.

In [ ]:
arraydata = np.reshape(np.array(djiaclean.rowid), (-1, 1))

trendmodel = LinearRegression().fit(arraydata, djiaclean.Close)
linearmodelpredictions = trendmodel.predict(arraydata)

(ggplot(djiaclean) +
  geom_line(mapping=aes('Date', 'Close')) +
  geom_line(mapping=aes('Date', linearmodelpredictions),color='red'))

In [ ]:
djiaclean["detrended"] = djiaclean["Close"] - linearmodelpredictions

(ggplot(djiaclean) +
  geom_line(mapping=aes('Date', 'detrended')))

In [ ]:
plots = []

for lag in difforders:
    plots.append((ggplot(djiaclean.dropna()) +
        geom_point(mapping=aes('Close', 'difforder'+str(lag))) +
        labs(title="Lagged Pickups of Order "+str(lag))))
plots

In [ ]:
maorders = [1, 3, 5, 7] ## Enter your orders here!

plots = []

for order in maorders:
    djiaclean['movingaverage'+str(order)] = djiaclean.Close.rolling(window=order).mean()
    plots.append((ggplot(djiaclean.dropna()) +
       geom_line(mapping=aes('Date','movingaverage'+str(order)))+
       labs(title=("Moving Average Difference Order: " + str(order)))))
    
plots

In [ ]:
plot_acf(djiaclean.Close, lags=500)
plot_pacf(djiaclean.Close, lags=10)
""

In [ ]:
djiatrain = djiaclean[:int(4*len(djiaclean)/5)]
djiatest = djiaclean[int(4*len(djiaclean)/5):]

In [ ]:
# Enter your orders below!

model = sarimax.SARIMAX(djiatrain.Close, order=(0, 1, 0))
result = model.fit()
predict_steps = len(djiatest)
djiatest["forecast"] = result.forecast(steps=predict_steps)

In [ ]:
print("MAPE: "+ str(100*np.mean(abs(djiatest.Close - djiatest.forecast)/djiatest.Close)))

(ggplot(djiatest) +
  geom_line(mapping=aes('Date', 'Close')) +
  geom_line(mapping=aes('Date', 'forecast'), color="red") +
  labs(title="A Fitted Arima Model", y="DJIA Close", x="Day"))

In [ ]:
prophet_train = djiatrain
prophet_test = djiatest

prophet_train = prophet_train.dropna()[["Date", "Close"]]
prophet_test = prophet_test.dropna()[["Date", "Close"]]

prophet_train.columns = ["ds", "y"]
prophet_test.columns = ["ds", "y"]

In [ ]:
m = Prophet(yearly_seasonality=False, daily_seasonality=True)
m.fit(prophet_train)

In [ ]:
future = m.make_future_dataframe(periods=len(djiatest), freq='D')
forecast = m.predict(future)

In [ ]:
prophetdf = pd.DataFrame({
    'Date': djiatest.Date,
    'Close': djiatest.Close,
    'Forecast': forecast.yhat
})

print("MAPE: "+ str(100*np.mean(abs(djiatest.Close - forecast.yhat)/djiatest.Close)))

(ggplot(prophetdf) +
  geom_line(mapping=aes('Date', 'Close')) +
  geom_line(mapping=aes('Date', 'Forecast'), color="red") +
  labs(title="A Fitted Prophet Model", y="DJIA Close", x="Day"))

In [ ]:
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM
from keras.layers.core import Dense

In [ ]:
num_days_back = 3

for day in range(1, num_days_back+1):
    djia['lag'+str(day)] = djia.Close.shift(day)

In [ ]:
colnames = list(map(lambda day: 'lag'+str(day + 1), list(range(num_days_back))))

djiatensor = np.array(djia[colnames])[num_days_back:, ]
djiatarget = np.array(djia.Close)[num_days_back:, ]

In [ ]:
cuts = len(djiatrain) - num_days_back

tf_train_x = djiatensor[:cuts].reshape((len(djiatrain) - num_days_back, num_days_back, 1))
tf_train_y = djiatarget[:cuts].reshape((len(djiatrain) - num_days_back, 1))
tf_test_x = djiatensor[cuts:].reshape((len(djiatest), num_days_back, 1))
tf_test_y = djiatarget[cuts:].reshape((len(djiatest), 1))

In [ ]:
rnn = Sequential([
    LSTM(units=300, input_shape=(num_days_back, 1), return_sequences=True, activation='relu'),
    LSTM(units=500, input_shape=(num_days_back, 1), return_sequences=True, activation='relu'),
    LSTM(units=300, input_shape=(num_days_back, 1), activation='relu'),
    Dense(200, activation='relu'),
    Dense(100, activation='relu'),
    Dense(1)
])

rnn.compile(optimizer="adam", loss="mean_absolute_percentage_error")
rnn.fit(tf_train_x, tf_train_y, epochs=5, validation_split=0.1)

In [ ]:
print("MAPE:" + str(rnn.evaluate(tf_test_x, tf_test_y)))

rnnpredictions = rnn.predict(tf_test_x)
predictionsdf = pd.DataFrame({"Prediction":list(rnnpredictions[:,0]), "Actual":djiatest.Close,
                              "Date":djiatest.Date})
(ggplot(predictionsdf) +
    geom_line(mapping=aes('Date', 'Prediction'), color='red') +
    geom_line(mapping=aes('Date', 'Actual')))